<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [8]</a>'.</span>

In [1]:
from utils import init_experiment
model_name = "bert-base-uncased"
init_experiment({"model": model_name})

wandb: Currently logged in as: kydlicek-hynek (hynky). Use `wandb login --relogin` to force relogin


wandb: wandb version 0.15.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


wandb: Tracking run with wandb version 0.15.3


wandb: Run data is saved locally in /lnet/aic/personal/kydliceh/o2/ml/wandb/run-20230607_013318-ds8ke0wx
wandb: Run `wandb offline` to turn off syncing.


wandb: Syncing run vital-surf-12


wandb: ⭐️ View project at https://wandb.ai/hynky/o2


wandb: 🚀 View run at https://wandb.ai/hynky/o2/runs/ds8ke0wx


In [2]:
import pandas as pd
train = pd.read_parquet("../artifacts/feature_store/yelp2020/train/")
test = pd.read_parquet("../artifacts/feature_store/yelp2020/test/")


In [3]:
# we will use small sample of the data 800k
train = train.sample(10000)
# split into train and validation
train = train.sample(frac=0.9, random_state=42)
val = train.drop(train.index)

In [4]:
from transformers import AutoTokenizer, PreTrainedTokenizer
from datasets import Dataset, ClassLabel
labels = test["usefulness"].unique().tolist()


def add_tokenization(dst: Dataset, tokenizer: PreTrainedTokenizer):
    def tokenize(batch):
        return tokenizer(batch["text"], padding="max_length", truncation=True)
    
    return dst.map(tokenize, batched=True, batch_size=1000)

def convert_to_hf_trainable_dataset(df: pd.DataFrame, tokenizer: PreTrainedTokenizer):
    dataset = Dataset.from_pandas(df)
    dataset = dataset.cast_column("usefulness", ClassLabel(names=labels))
    dataset = dataset.rename_column("usefulness", "labels")
    dataset = add_tokenization(dataset, tokenizer)
    return dataset

/lnet/aic/personal/kydliceh/o2/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

train_dataset = convert_to_hf_trainable_dataset(train, tokenizer)
val_dataset = convert_to_hf_trainable_dataset(val, tokenizer)
test_dataset = convert_to_hf_trainable_dataset(test, tokenizer)

Casting the dataset:   0%|                                                                                                                                           | 0/9000 [00:00<?, ? examples/s]

Map:   0%|                                                                                                                                                           | 0/9000 [00:00<?, ? examples/s]

Map:  11%|███████████████▊                                                                                                                              | 1000/9000 [00:00<00:05, 1509.77 examples/s]

Map:  22%|███████████████████████████████▌                                                                                                              | 2000/9000 [00:01<00:04, 1452.14 examples/s]

Map:  33%|███████████████████████████████████████████████▎                                                                                              | 3000/9000 [00:02<00:04, 1447.10 examples/s]

...

Map:  99%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌ | 347000/350647 [03:59<00:02, 1477.09 examples/s]

Map:  99%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉ | 348000/350647 [04:00<00:01, 1505.79 examples/s]

Map: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎| 349000/350647 [04:00<00:01, 1511.37 examples/s]

Map: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋| 350000/350647 [04:01<00:00, 1500.43 examples/s]

Map: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 350647/350647 [04:02<00:00, 1485.11 examples/s]

In [6]:
from transformers import AutoModelForSequenceClassification

def freeze_all_but_classifier_and_last_layers(model, num_layers):
    for param in model.base_model.parameters():
        param.requires_grad = False

    for layer in model.base_model.encoder.layer[-num_layers:]:
        for param in layer.parameters():
            param.requires_grad = True
    
    model.classifier.requires_grad = True
    return model

def get_model_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

transformer = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(labels))
transformer = freeze_all_but_classifier_and_last_layers(transformer, 1)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
get_model_params(transformer)

7090179

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [8]:
from transformers import TrainingArguments
from transformers import Trainer
from utils import get_result, get_model_folder

training_args = TrainingArguments(
    output_dir=get_model_folder(),
    num_train_epochs=1,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    logging_steps=10,
    report_to="wandb",
)



compute_metrics = lambda pred: {"f1_macro": get_result(pred.label_ids, pred.predictions.argmax(-1)).metric_value}
trainer = Trainer(
    model=transformer,
    args=training_args,
    train_dataset=train_yelp_dataset,
    eval_dataset=val_yelp_dataset,
    compute_metrics=compute_metrics,
)

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate`: Run `pip install --upgrade accelerate`

In [ ]:
trainer.train()